In [1]:
import numpy as np
import math
import scipy
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from scipy.stats import qmc
from sklearn.gaussian_process.kernels import ConstantKernel
import fusionrate
from fusionrate import Reaction
from scipy.constants import eV
import random

In [2]:
from dataclasses import dataclass

In [3]:
C_SCALE = 1e-20

In [4]:
CUBIC_CM_PER_CUBIC_M = 1e6

In [5]:
MeV = eV * 1e6

In [6]:
REACTION_TO_ENERGY = 17.58 * MeV

In [7]:
@dataclass
class TrainedData:
    
    #these will be running "totals" of all training data 
    x_train: list #inputs (alpha_n, alpha_T, T0)
    y_train: list #outputs (c, B)

td = TrainedData(list(),list())

In [8]:
class FRCounter:
    
    low_fidelity_model = 0
    high_fidelity_model = 0


In [9]:
def generate_profile(rho, x0, alpha_x):
    """This function generates radial based parabolic profiles 
    for densities / temperatures , 0<rho<1

    args
    ----------
    - rho (array-like): radial position
    - n0 (float): denisity / temperature at rho = 0 
    - alpha_x (float): exponential value predicted between 1.00 - 3.00 

    returns
    ----------
    - profile (array-like): an array of radial based profiles (temp or density)

    """
    profile = x0 * ((1 - (rho**2))**alpha_x)

    return profile 

In [10]:
def volume_averaged_density(densities, volumes):
    """This function finds the volume averaged density 

    Parms:
        - densisties (array-like): an array of densities of each partition
            #/m^3
        - volumes (array-like): an array of volumes of each parition
            m^3

    Returns:
        - vol_avg_density (float): density averaged over the whole volume
            #/m^3
    """
    vol_avg_density = (np.sum(np.multiply(densities,volumes)))/(np.sum(volumes))

    return vol_avg_density

In [11]:
def volume_averaged_temperature(densities, volumes, temperatures):
    """This function finds the volume averaged temperature 

    args
    -------
        - densities(array-like): an array of densities of each partition
            #/m^3
        - volumes(array-like): an array of volumes of each parition
            m^3
        - temperature(array-like): an array of temperatures of each partition 
            KeV

    return
    ---------
        - vol_avg_temperature (float): temperature averaged over the whole volume 
            Kev

    """
    vol_avg_temperature = (np.sum(np.multiply(temperatures,(np.multiply(densities,volumes))))/(np.sum(np.multiply(densities,volumes))))
    type(vol_avg_temperature)

    return vol_avg_temperature

In [12]:
def generate_volumes(rho, a, R):
    '''This function creates an array of volume elements of equal temperature and density

    args
    ------
    rho (array-like): array of radii 
        meters
    a (float): minor radius of torus
        meters
    R (float): major radius of torus 
        meters

    return
    -------
    volumes (array-like): array of volume elements 
        m^3
    '''
    drho = 1/len(rho) #the change in rho is the amount of lengths (1/100th in this case)

    #initialzing
    total_area = 0 

    volumes = []

    for i in rho: #indexing elements of array rho
        ring_area = (a**2) * i * drho * 2*math.pi 
        ring_volume = ring_area *2*math.pi * R
        volumes.append(ring_volume)

    return volumes

In [13]:
def initialize_training_data(n0, a, R):
    ''' Initializes variable x_train, calls generated halton sequence and adds to x_train
    '''
    td.x_train = []
    td.y_train = []
    
    n = 5 #number of training points 
    ranges = [(1,5), (1,5), (1,100)] #ranges for alpha_n, alpha_T, and T0
    sequence = generate_halton_sequence(n, ranges) #call generate_halton_sequence for number of training points and ranges 

    #ensure 'sequence' is a list
    td.x_train.extend(sequence) #adds sequence to x_train list 

    #generate corresponding y_train from x_train (generate_lowfidelityFR )

    for x in td.x_train:
        alpha_n, alpha_T, T0 = x 
        y = generate_lowfidelityFR(n0, alpha_n, T0, alpha_T, a, R)
        td.y_train.append(y) #modify y_train to include new training points
        

In [14]:
def generate_test_data(num, range1, range2, range3):
    """
    Generate a number of 3-element long arrays with random numbers.

    Parameters:
        num(int): The number of arrays to be generated.
        range1 (tuple): A tuple (min_value, max_value) specifying the range for the first element.
        range2 (tuple): A tuple (min_value, max_value) specifying the range for the second element.
        range3 (tuple): A tuple (min_value, max_value) specifying the range for the third element.

    Returns:
        List of tuples: A list containing the randomly generated tuples.
    """
    result = []
    for _ in range(num):
        element1 = random.uniform(element1_range[0], element1_range[1])
        element2 = random.uniform(element2_range[0], element2_range[1])
        element3 = random.uniform(element3_range[0], element3_range[1])
        result.append((element1, element2, element3))
    return result

# Example usage:
num = 10_000
element1_range = (1, 5)
element2_range = (1, 5)
element3_range = (1, 100)

x_test = generate_test_data(num, element1_range, element2_range, element3_range)
print(x_test)



[(3.0606280708504676, 2.7806151998954456, 15.387405660203985), (4.125859508007634, 1.7867989071734893, 3.6925639221936493), (1.642700338447841, 1.9807067751271692, 78.36548640941025), (3.7086723175954805, 3.5776594213016426, 18.314295100529797), (4.047275704609061, 2.207950505973333, 16.334388511390515), (1.6134605617977833, 4.487671227603956, 40.27594016384568), (2.229889457160537, 1.5786900722884507, 46.76295674556886), (2.932630913999489, 2.9933668088399585, 68.88825021957142), (4.502334336687587, 3.6379785793570907, 67.79227914048518), (4.774958794929521, 4.084595459165518, 44.41531982535841), (1.4484611843097435, 2.539516167886568, 72.20932760651596), (4.345242996582606, 3.9025055895014997, 18.334596020993647), (3.3625199038994236, 2.921758547778428, 1.855378224102707), (1.5486052253965306, 1.8522196591120785, 87.70180079055069), (4.720055448441265, 4.049548665938092, 2.3817451140003216), (1.6594707204204595, 4.22421133402473, 72.66354150516351), (1.1979573303168691, 2.95964956098

In [15]:
#Fusion Rate Models : 
def compute_highfidelityFR(n0, alpha_n, T0, alpha_T, a, R):
    '''This function computes the high-fidelity fusion rate

    args
    ------
    - n0: initial density, estimate around 10**20 (float) 
        #/m^3
    - alpha_n: exponential (flot)
        #/m^3
    - T0: initial temperature, estimate around 10 Kev (float)
        KeV
    - alpha_T: exponential (float)
        KeV
    - a: torus minor radius (float)
        meters
    - R: torus major radius (float)
        meters

    returns
    --------
    - highfidelityFR: high-fidelity fusion rate (float)
        # reactions / m^3 * s 

    '''
    rho = np.linspace(0,1,num=100) 

    densities = generate_profile(rho,n0,alpha_n)

    temperatures = generate_profile(rho,T0,alpha_T)

    volumes = generate_volumes(rho,a,R)

    dt = Reaction('D+T')
    rate_coeff = dt.rate_coefficient(temperatures, scheme = 'analytic')
    
    rate_coeff = rate_coeff / CUBIC_CM_PER_CUBIC_M

    sectionalHFFR = (np.multiply(rate_coeff,np.multiply(densities,densities))) 

    highfidelityFR=sum(np.multiply(sectionalHFFR,volumes))
    
    FRCounter.high_fidelity_model+= 1
    
    return highfidelityFR

In [16]:
def compute_highfidelityFR_deriv(n0, alpha_n, T0, alpha_T, a, R):
    '''This function computes the derivative of the high-fidelity fusion rate with respect to temperature

    args
    ------
     - n0: initial density, estimate around 10**20 (float) 
        #/m^3
    - alpha_n: exponential (flot)
        #/m^3
    - T0: initial temperature, estimate around 10 Kev (float)
        KeV
    - alpha_T: exponential (float)
        KeV
    - a: torus minor radius (float)
        meters
    - R: torus major radius (float)
        meters


    returns
    --------
    - highfidelityFR_deriv: derivative of high fidelity fusion rate (float)
       # reactions / m^3 * s   

    '''
    rho = np.linspace(0,1,num=100)

    densities = generate_profile(rho,n0,alpha_n)

    temperatures = generate_profile(rho,T0,alpha_T)

    volumes = generate_volumes(rho,a,R)

    dt = Reaction("D+T")
    rate_coeff_deriv = dt.rate_coefficient(temperatures, scheme ='analytic',derivatives=True)

    rate_coeff_deriv = rate_coeff_deriv / CUBIC_CM_PER_CUBIC_M
    
    sectionalHFFR_deriv = (np.multiply(rate_coeff_deriv,np.multiply(densities,densities)))

    highfidelityFR_deriv=sum(np.multiply(sectionalHFFR_deriv,volumes))

    return highfidelityFR_deriv

In [17]:
def compute_lowfidelityFR(n0, alpha_n, T0, alpha_T, a, R, c, B):
    """This function computes the low-fidelity fusion rate

    args
    ------
    -n0: initial density, estimate around 10**20 (float) 
         #/m^3
    - alpha_n: exponential (float)
         #/m^3
    - T0: initial temperature, estimate around 10 Kev (float)
         KeV
    - alpha_T: exponential (float)
        KeV
    - a: torus minor radius (float)
        meters
    - R: torus major radius (float)
        meters
    - c: (float)
        m^3 / s
    - B: (float)
        dimensionless

    returns
    --------
    - lowfidelityFR: low-fidelity fusion rate (float)
       # reactions / m^3 * s    

    """
    rho = np.linspace(0,1,num=100)

    densities = generate_profile(rho,n0,alpha_n)
    temperatures = generate_profile(rho,T0,alpha_T)
    volumes = generate_volumes(rho,a,R)

    total_volume = sum(volumes)

    vol_avg_density = volume_averaged_density(densities, volumes)
    vol_avg_temperature = volume_averaged_temperature(densities, volumes, temperatures)


    lowfidelityFR = total_volume * C_SCALE * c * vol_avg_density**2 *vol_avg_temperature**B
    
    FRCounter.low_fidelity_model += 1

    return lowfidelityFR

In [18]:
def compute_lowfidelityFR_deriv(n0, alpha_n, T0, alpha_T, a, R, c, B):
    """This function computes the derivative of the high-fidelity fusion rate with respect to temperature

     args
    ------
     -n0: initial density, estimate around 10**20 (float) 
         #/m^3
    - alpha_n: exponential (float)
         #/m^3
    - T0: initial temperature, estimate around 10 Kev (float)
         KeV
    - alpha_T: exponential (float)
        KeV
    - a: torus minor radius (float)
        meters
    - R: torus major radius (float)
        meters
    - c: (float)
        m^3 / s
    - B: (float)
        dimensionless

    returns
    ---------
    -lowfidelityFR_deriv: derivative of low-fidelity fusion rate (float)
         # reactions / m^3 * s    

    """
    rho = np.linspace(0,1,num=100)

    densities = generate_profile(rho,n0,alpha_n)
    temperatures = generate_profile(rho,T0,alpha_T)
    volumes = generate_volumes(rho,a,R)

    total_volume = sum(volumes)

    vol_avg_density = volume_averaged_density(densities, volumes)
    vol_avg_temperature = volume_averaged_temperature(densities, volumes, temperatures)

    lowfidelityFR_deriv = total_volume * C_SCALE * c * vol_avg_density**2 * B * vol_avg_temperature**(B-1)

    return lowfidelityFR_deriv

In [19]:
def generate_lowfidelityFR(n0, alpha_n, T0, alpha_T, a, R):
    '''This function generates parameters necessary for computation of the low-fidelity FR model

   args
    ------
      - n0: initial density, estimate around 10**20 (float) 
        #/m^3
    - alpha_n: exponential (flot)
        #/m^3
    - T0: initial temperature, estimate around 10 Kev (float)
        KeV
    - alpha_T: exponential (float)
        KeV
    - a: torus minor radius (float)
        meters
    - R: torus major radius (float)
        meters


    returns
    --------
    - c: compute best guess from formula (float)
        m^3 / s
    - B: best guess around 2 (float)
        dimensionless
    '''
    rho = np.linspace(0,1,num=100)

    highfidelityFR = compute_highfidelityFR(n0, alpha_n, T0, alpha_T, a, R)
    highfidelityFR_deriv = compute_highfidelityFR_deriv(n0, alpha_n, T0, alpha_T, a, R)

    densities = generate_profile(rho,n0,alpha_n)
    temperatures = generate_profile(rho,T0,alpha_T)
    volumes = generate_volumes(rho,a,R)

    total_volume = sum(volumes)

    vol_avg_density = volume_averaged_density(densities, volumes)
    vol_avg_temperature = volume_averaged_temperature(densities, volumes, temperatures)

    def f1(c,B):
        return total_volume * c * vol_avg_density**2 *vol_avg_temperature**B - highfidelityFR / C_SCALE

    def f2(c,B):
        return total_volume * c * vol_avg_density**2 * B * vol_avg_temperature**(B-1) - highfidelityFR_deriv / C_SCALE

    def f(c,B):
        return f1(c,B), f2(c,B)

    def wrapper_of_f_for_fsolve(x):
        return np.array(f(*x))

    c_sol = scipy.optimize.fsolve(wrapper_of_f_for_fsolve, [1,2], xtol=1e-2)
    
    return c_sol #c_sol will need to be indexed as c = c_sol[0], B = c_sol[1], built to return c and B

In [20]:
generate_lowfidelityFR(1, 2, 100, 2, 1, 10)

array([0.00491082, 0.84067946])

In [21]:
def x_test_from_generic_inputs(n0, alpha_n, T0, alpha_T, a, R):
    ''' converts physics inputs into a x_test point 
    Args
    --------
      - n0: initial density, estimate around 10**20 (float) 
        #/m^3
    - alpha_n: exponential (flot)
        #/m^3
    - T0: initial temperature, estimate around 10 Kev (float)
        KeV
    - alpha_T: exponential (float)
        KeV
    - a: torus minor radius (float)
        meters
    - R: torus major radius (float)
        meters

    Returns
    ---------
    x_test (array-like): 1d np.array of alpha_n, alpha_T, T0
    (#/m^3, KeV, KeV)

    '''
    
    x_test = (alpha_n, alpha_T, T0) #not sure if this is correct or correct order 
    return x_test

In [22]:
def generate_halton_sequence(n, ranges):
    """
    This function generates the training points for data over interesting ranges

    Arguments:
    - n: The number of points to generate.
    
    - ranges: A list of tuples specifying the range for each dimension.
        range in m 

    Returns:
    A list of tuples representing the generated points in the Halton sequence.
    """
    
    num_dimensions = len(ranges)
    low = np.array([min_value for min_value, _ in ranges])
    high = np.array([max_value for _, max_value in ranges])
    halton = qmc.Halton(num_dimensions)
    scaled_points = halton.random(n)
    scaled_points = low + scaled_points * (high - low)
    sequence = [tuple(point) for point in scaled_points]

    return sequence

In [23]:
def add_training_points(new_x_train, new_y_train):
    # may need some conversions 

    td.x_train.append(new_x_train)

    td.y_train.append(new_y_train)

In [24]:
def evaluate_values(x_train, y_train, x_test):
    '''
    This function returns the prediction for c and B given a 2D array test point 

    args
    ------
    X_train (array-like) : training data - "randomly" generated on interesting bounds
    y_train (array- like) : produced in generate low fidelity function returning associated c and B values 
    X_test (array-like) : point for uncertainty to be evauluated at (2d array)

    returns
    --------
    yc_std (float): The prediction of c at X_test
        m^3/s
    yB_std (float): The prediction deviation of B at X_test
        dimensionless 

    '''
    x = np.array(td.x_train)
    yc,yB = np.array(td.y_train).T

    #defining the kernel for c then B
    kernelc = ConstantKernel(constant_value=1.0,constant_value_bounds=(1e-5,1e3)) * RBF(length_scale=1, length_scale_bounds=(1e-2,100))   
    kernelB = ConstantKernel(constant_value=1.0,constant_value_bounds=(.1,100)) * RBF(length_scale=0.5, length_scale_bounds=(0.1,100))   

    #create a GPR model
    gprc = GaussianProcessRegressor(kernel=kernelc, n_restarts_optimizer=9)
    gprB = GaussianProcessRegressor(kernel=kernelB, n_restarts_optimizer=9)
    

    #fit model to data
    gprc.fit(x, np.log(yc))
    gprB.fit(x,yB)


    #generate points in inputspace to predict 
    x_pred = np.linspace(0,5,100)

    #predict function values for input points & standard deviation 
    yc_pred= gprc.predict(x_test, return_std = False)[0]
    yB_pred = gprB.predict(x_test, return_std = False)[0]


    #print("The standard deviation of c at", point, "is", yc_std*10e-16, "(log units)")
    #print("The standard deviation of B at", point, "is", yB_std)

    return yc_pred, yB_pred

In [25]:
def evaluate_uncertainty(x_train, y_train, x_test):
    '''
    This function returns the uncertainty/ error/ standard dev for c and B given a 2D array test point 

    args
    ------
    X_train (array-like) : training data - "randomly" generated on interesting bounds
    y_train (array- like) : produced in generate low fidelity function returning associated c and B values 
    X_test (array-like) : point for uncertainty to be evauluated at (2d array)

    returns
    --------
    yc_std (float): The standard deviation of c at X_test
        m^3 / s
    yB_std (float): The standard deviation of B at X_test
        dimensionless

    '''
    x = np.array(td.x_train )
    
    yc,yB = np.array(y_train).T

    #defining the kernel for c then B
    kernelc = ConstantKernel(constant_value=1.0,constant_value_bounds=(1e-5,1e3)) * RBF(length_scale=1, length_scale_bounds=(1e-2,100))   
    kernelB = ConstantKernel(constant_value=1.0,constant_value_bounds=(.1,100)) * RBF(length_scale=0.5, length_scale_bounds=(0.1,100))   

    #create a GPR model
    gprc = GaussianProcessRegressor(kernel=kernelc, n_restarts_optimizer=9)
    gprB = GaussianProcessRegressor(kernel=kernelB, n_restarts_optimizer=9)

    #fit model to data
    gprc.fit(x, np.log(yc))
    gprB.fit(x,yB)

    #generate points in inputspace to predict 
    x_pred = np.linspace(0,5,100)

    #predict function values for input points & standard deviation 
    yc_std = gprc.predict(x_test, return_std = True)[1][0]
    yB_std = gprB.predict(x_test, return_std = True)[1][0]
    
    

    #print("The standard deviation of c at", point, "is", yc_std*10e-16, "(log units)")
    #print("The standard deviation of B at", point, "is", yB_std)

    return yc_std, yB_std

In [26]:
def test_uncertainty(std_devs, B, max_uncertainty):
    '''The purpose of this function is to test if the uncertainty is within bounds of the max uncertainty
    args
    -----
    std (tuple): tuple of errors for c and B where the first is taken to be in logspace and the second in linear space
    B (float): point of B 
    max_uncertainty): tuple of associated allowed errors as a fraction 
    
    *** c is taken to be in logspace, B is taken to be in linear sapce
    returns
    ---------
    bool

    '''
    
    c_std_dev_log = std_devs[0]
    c_std_dev = math.exp(c_std_dev_log)
    c_close_enough = c_std_dev <= max_uncertainty + 1
    
    B_std_dev = std_devs[1]
    B_frac_uncert =  B_std_dev / B
    B_close_enough = B_frac_uncert <= max_uncertainty
    
    #max uncert may need to be a tuple - can't set one specific uncert for both ex 20% of accepted value
    return c_close_enough and B_close_enough
                

In [27]:
from icecream import ic

In [28]:
std_devs = [0.01, 0.2]
B = 1
max_uncertainty = 0.2
test_uncertainty(std_devs, B, max_uncertainty)

True

In [29]:
def get_a_point(n0, alpha_n, T_0, alpha_T, a, R, uncertainty):
    '''This function takes in a point and alloted uncertainty and returns the fusion rate 

    Args
    ----
     -n0: initial density, estimate around 10**20 (float) 
    - alpha_n: exponential (float)
    - T0: initial temperature, estimate around 10 Kev (float)
    - alpha_T: exponential (float)
    - a: torus minor radius (float)
    - R: torus major radius (float)
    - uncertainty (float) 

    Returns
    --------
    - Total fusion rate

    '''

    x_test = x_test_from_generic_inputs(n0, alpha_n, T0, alpha_T, a, R) # create test points 
    x_test =np.array(x_test)
    x_test = x_test[None, : ]

    c_log, B = evaluate_values(td.x_train, td.y_train, x_test) # use the GPR to find corresponding prediction values for x_test based on trained data
    c = math.exp(c_log)
    
    yc_std, yB_std = evaluate_uncertainty(td.x_train, td.y_train, x_test) #evaluate the uncertainty of those predicted values 

    
    
    std = (yc_std, yB_std) 

    is_acceptable = test_uncertainty(std, B, uncertainty)

    if is_acceptable:
        #use c and B to evaluate lfm - the error is minimal enough to accept LFM 

            pass 
    else:
        #The error is too large to accept lfm - 
        c, B = generate_lowfidelityFR(n0, alpha_n, T0, alpha_T, a, R) #use the HFM to calculate c,B 
        new_y_train = c, B

        x_test = x_test[0].tolist()
        add_training_points(x_test, new_y_train) #add those to new training data
        #find fusion rate based on c & B
            
    total_fusion_rate = compute_lowfidelityFR(n0, alpha_n, T0, alpha_T, a, R, c, B)
        


In [30]:
initialize_training_data(10**12, 1, 10)

In [31]:
n0= 10**20
# alpha_n= 2
# T0=100
# alpha_T= 2
a= 2
R= 6.2
uncertainty = .3
num = 10
range1 = (1, 5)
range2 = (1, 5)
range3 = (1, 100)
test_data = generate_test_data(num, range1, range2, range3)

for row in test_data:
    print(row)
    alpha_n, alpha_T, T0 = row 
    get_a_point(n0, alpha_n, T0, alpha_T, a, R, uncertainty)

(4.699801251927983, 3.0368542078231133, 42.25794303859839)
(3.3780476406524724, 3.7256640721066607, 90.18992987397291)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


(1.249815362141533, 2.3708638148160635, 55.97759930153231)
(2.363442459567825, 1.5768633786946498, 16.094606786372648)
(1.952494580924192, 4.808495276064919, 67.32908798182363)
(2.5072772085049135, 4.614861364624285, 71.52878713276378)
(4.857367794875268, 3.89223559561467, 94.7283754273999)
(1.0012781614901498, 4.9848843076787155, 3.399727610919848)
(2.9691726328123975, 4.770218644369703, 57.49647197976722)
(2.923495441504805, 3.4143691894031774, 70.9219077545083)


In [32]:
print(FRCounter.low_fidelity_model)

10


In [33]:
FRCounter.high_fidelity_model

12

In [34]:

len(td.x_train)

11

In [35]:
len(td.y_train)

11

In [36]:
n_grid = 200 #sets number of points in each grid to 200

alpha_n_test = np.linspace(1,5, num =n_grid) #200 points between 1 and 5
alpha_T_test = np.linspace(1,5,num =n_grid) #200 points between 1 and 5 
#T0_test=np.geomspace(math.log(1.0000000001),math.log(100),num =n_grid) #logarithmically spaced values 
T0_test=np.linspace(math.log(1.0000000001),math.log(100),num =n_grid)
alpha_T_grid, T0_grid = np.meshgrid(alpha_T_test, T0_test) #creates a grid of points from alpha_T_test and T0 test
alpha_n_grid = np.ones_like(alpha_T_grid)*2.0 #array filled with 2s


In [37]:
#reshapes 3 grids into an array "test_points"
test_points = np.reshape(np.array([alpha_T_grid,alpha_n_grid, T0_grid]),(3,n_grid*n_grid)).T

In [39]:
y1_pred, y1_std = evaluate_values(test_points, return_std = True)
y1_pred = np.reshape(y1_pred,(n_grid,n_grid))
y1_std = np.reshape(y1_std,(n_grid,n_grid))

TypeError: evaluate_values() got an unexpected keyword argument 'return_std'

In [ ]:
fig1, ax = plt.subplots()
ax.set_yscale('log')
plt.xlabel(r"$\alpha_T$", size = 20)
plt.ylabel(r"$T_0$ / keV", size = 20)
CS = ax.contourf(alpha_T_grid, T0_grid, y1_pred)
#ax.clabel(CS, inline= True, fontsize = 10)
plt.axis([1,5,1,100])
ax.set_title('Predicted Value ' + r'$\log(c * 10^{16} \; \mathrm{m}^{-3}\mathrm{s})$', size = 20)

ax.tick_params(axis='both', which='major', labelsize=15)

cbar = fig1.colorbar(CS)
cbar.ax.set_ylabel('c Prediction', size = 20)

In [ ]:
X1 = np.sort(np.array(td.x_train).T[0])

In [ ]:
print(X) #these are all alpha_n values 

In [ ]:
y1 = np.sort(np.array(td.y_train).T[1])

In [ ]:
print(y) #these are all B values 

In [ ]:
X1_train = np.sort(np.array(td.x_train).T[0])
y1_train = np.sort(np.array(td.y_train).T[1])

In [ ]:
X1_train 

In [ ]:
y1_train

In [ ]:
x_train = td.x_train
y_train = td.y_train
mean_predictions = evaluate_values(x_train, y_train, x_test)

In [ ]:
mean_predictions

In [ ]:
fig, axs = plt.subplots(1)
plt.plot(X1,y1,linestyle="dotted")
axs.set_xlabel("alpha_n")
axs.set_ylabel("Beta")

plt.scatter(X1_train, y1_train, label="Observations")

plt.plot(X1, mean_prediction[1], label="Mean prediction")

In [ ]:
n0= 10**20
# alpha_n= 2
# T0=100
# alpha_T= 2
a= 2
R= 6.2
uncertainty = .2
num = 7
range1 = (1, 5)
range2 = (1, 5)
range3 = (1, 100)
test_data = generate_test_data(num, range1, range2, range3)

for row in test_data:
    alpha_n, _, _ = row 
    

r'$\rho$''

$\left<\right>$

$$\int_0^1 \frac{dV}{d\rho}(\rho)\left<\sigma v \right>(T)\,n^2d\rho$$

In [ ]:
profile = x0 * ((1 - (rho**2))**alpha_x)

$$x(\rho) \doteq x_0 (1- \rho^2)^{\alpha_x}$$